In [1]:
# libraries and packages
! pip install crewai crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
! pip show crewai

Name: crewai
Version: 0.35.8
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: https://crewai.com
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: appdirs, click, embedchain, instructor, jsonref, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import crewai
from crewai import Agent, Task, Crew

In [5]:
# Working with ChatOpenAI from LangChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI Chat model
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

# Send a message to the model and get the response
response = llm.invoke("what is the capital of India?")
print(response.content)

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


The capital of India is New Delhi.


In [6]:
#1- Chef
chef = Agent(
    role="chef",
    goal="Give 5 different receipes using ingredients : {ingredients}",
    backstory="You're  a Chef who has experience on cooking delicious foods"
              "Your expertice lies in recommeding some new foods to the customers"
              "Please generate and give receipe of 5 different foods using this ingredients : {ingredients} "
              "describing stepwise process, time to cook, other ingredients needed and calorie count"
              "Your recipe will be critically analyzed by nutritionist to recommend food for diabetes patients ",
	verbose=True
)

In [7]:
2# Nutritionist
nutritionist = Agent(
    role="nutritionist",
    goal="Critically analyze the recipe given by chef using these ingredients: {ingredients}",
    backstory="You're a nutritionist and have good knowledge on working with Diabetic patients "
              "There are recipe recommended by chef using these ingredients : {ingredients}. "
              "please critically analyze the recipe and suggest best food out of given 5 for Diabetic patients"
              "Also give reasons why you dont recommend other 4 receips and choose this one",
    verbose=True
)

In [8]:
cook = Task(
    description=(
        "1. Write receipe of 5 different foods using these ingredients  : {ingredients}.\n"
    ),
    expected_output="5 different receipes using {ingredients}"
                    "Stepwise guide to cook"
                    "List of all additional ingredients required for cooking"
                    "The expected taste of the food",
    agent=chef,
)

In [9]:
recommend = Task(
    description=(
        "1. Read the receipes provided by chef using {ingredients} \n"
        "2. critically analyze all receipes keeping in mind what should suit to diabetic patients.\n"
        "3. Suggest one best food out of all 5 receipes given by chef.\n"
        "4. Prove that other 4 receipes suggested by chef is not good for diabetic patients.\n"
        
    ),
    expected_output="A well-written critical pointwise analysis of all 5 recipes using {ingredients}"
                    "Suggest one receipe to diabetic patients based on your own knowledge and evidence "
                    "Reject all other 4 receipes with knowledge and evidence "
                    "Finally, give the stepwise receipe of what you selected and a nice message",
    agent=nutritionist,
)

In [10]:
crew_run = Crew(
    agents=[chef,nutritionist],
    tasks=[cook, recommend],
    verbose=2
)

In [12]:
result = crew_run.kickoff(inputs={"ingredients": "spinach"})

 [2025-06-21 23:44:34][DEBUG]: == Working Agent: chef
 [2025-06-21 23:44:34][INFO]: == Starting Task: 1. Write receipe of 5 different foods using these ingredients  : spinach.



> Entering new CrewAgentExecutor chain...
To generate five different recipes using spinach, I should first create the recipes including a stepwise guide for cooking, a list of additional ingredients, and an expected taste description. Then, I will delegate the task of analyzing these recipes for suitability for diabetes patients to the nutritionist. 

Action: Delegate work to coworker
Action Input: {"task": "Analyze the suitability of the following spinach-based recipes for diabetes patients", "context": "1. Spinach and Mushroom Frittata: Ingredients: eggs, spinach, mushrooms, onion, garlic, olive oil, salt, pepper. Steps: Sauté onions, garlic, and mushrooms in olive oil, add spinach until wilted, pour beaten eggs over the mixture, cook until set. Taste: Savory and rich. 2. Spinach and Lentil Soup: Ingredients

### Phase 1: Chef Agent Starts

- **Initial Action**: Chef begins generating spinach recipes
- **Early Delegation Attempt**: Chef tries to ask nutritionist for help (too early!)
- **Error Recognition**: System detects that nutritionist doesn't have recipes yet
- **Self-Correction**: Chef realizes it needs to complete recipes first


### Phase 2: Recipe Creation

- **Recipe 1**: Spinach and Feta Salad
- **Recipe 2**: Creamed Spinach with Garlic
- **Recipe 3**: Spinach and Mushroom Stir-fry
- **Recipe 4**: Spinach Soup with Lentils
- **Recipe 5**: Spinach and Ricotta Stuffed Chicken


### Phase 3: Nutritionist Analysis

- **Input Reception**: Receives all 5 completed recipes
- **Individual Analysis**: Evaluates each recipe for diabetic suitability
- **Comparison**: Weighs pros and cons of each option
- **Final Recommendation**: Selects best recipe with detailed reasoning